In [ ]:
!pip install transformers
!pip install pytorch-transformers

In [ ]:
import numpy as np
import regex as re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
import math
import os

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow.keras.backend as K
import tokenizers
from transformers import XLMTokenizer, TFXLMModel, XLMForSequenceClassification, XLMConfig

from collections import Counter

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Detect hardware, return appropriate distribution strategy (you can see that it is pretty easy to set up).
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is set (always set in Kaggle)
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    print('Running on TPU ', tpu.master())
except ValueError:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print('Number of replicas:', strategy.num_replicas_in_sync)

INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.102.164.18:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.102.164.18:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Running on TPU  grpc://10.102.164.18:8470
Number of replicas: 8


In [ ]:
#MODEL_NAME = 'xlm-mlm-en-2048'
MODEL_NAME = 'xlm-mlm-ende-1024'
MAX_LEN = 256
ARTIFACTS_PATH = '../artifacts/'

BATCH_SIZE = 4 * strategy.num_replicas_in_sync
EPOCHS = 3

if not os.path.exists(ARTIFACTS_PATH):
    os.makedirs(ARTIFACTS_PATH)

In [ ]:
! gdown --id 1etLBrBTdokVHIuaxEmr1Koacos4IbMZL ## ConsumerComplaints

Downloading...
From: https://drive.google.com/uc?id=1etLBrBTdokVHIuaxEmr1Koacos4IbMZL
To: /content/ProcessedConsumerData.csv
2.24GB [00:17, 128MB/s] 


In [ ]:
import pandas as pd
import io
data = pd.read_csv('ProcessedConsumerData.csv')

In [ ]:
##  Creating a new Dataframe with the cols we're interested In

cols = ['TokenizedText', 'Product']
data = data[cols]
data.head(10)

,TokenizedText,Product
0,transworld systems inc trying collect debt min...,Debt collection
1,past weeks receiving excessive amounts telepho...,Debt collection
2,pioneer committed several federal violations p...,Debt collection
3,previously requested experian send copy verifi...,"Credit reporting, credit repair services, or o..."
4,hello complaint three credit reporting compani...,"Credit reporting, credit repair services, or o..."
5,today went online dispute incorrect personal i...,"Credit reporting, credit repair services, or o..."
6,reporting incorrectly equifax account balance ...,"Credit reporting, credit repair services, or o..."
7,please reverse late payments reported followin...,"Credit reporting, credit repair services, or o..."
8,victim identity theft previously stated,"Credit reporting, credit repair services, or o..."
9,experian reporting inaccurate false informatio...,"Credit reporting, credit repair services, or o..."


In [ ]:

X_data = data[['TokenizedText']].to_numpy().reshape(-1)
y_data = data[['Product']].to_numpy().reshape(-1)

In [ ]:
def xlm_encode(texts, tokenizer):
    ct = len(texts)
    input_ids = np.ones((ct, MAX_LEN), dtype='int32')
    attention_mask = np.zeros((ct, MAX_LEN), dtype='int32')
    token_type_ids = np.zeros((ct, MAX_LEN), dtype='int32') # Not used in text classification

    for k, text in enumerate(texts):
        # Tokenize
        tok_text = tokenizer.tokenize(str(text))

        # Truncate and convert tokens to numerical IDs
        enc_text = tokenizer.convert_tokens_to_ids(tok_text[:(MAX_LEN-2)])

        input_length = len(enc_text) + 2
        input_length = input_length if input_length < MAX_LEN else MAX_LEN

        # Add tokens [CLS] and [SEP] at the beginning and the end
        input_ids[k,:input_length] = np.asarray([0] + enc_text + [2], dtype='int32')

        # Set to 1s in the attention input
        attention_mask[k,:input_length] = 1

    return {
        'input_word_ids': input_ids,
        'input_mask': attention_mask,
        'input_type_ids': token_type_ids
    }

In [ ]:
# Transform categories into numbers
category_to_id = {}
category_to_name = {}

for index, c in enumerate(y_data):
    if c in category_to_id:
        category_id = category_to_id[c]
    else:
        category_id = len(category_to_id)
        category_to_id[c] = category_id
        category_to_name[category_id] = c

    y_data[index] = category_id

# Display dictionary
category_to_name

{0: 'Debt collection',
 1: 'Credit reporting, credit repair services, or other personal consumer reports',
 2: 'Money transfer, virtual currency, or money service',
 3: 'Mortgage',
 4: 'Vehicle loan or lease',
 5: 'Student loan',
 6: 'Credit card or prepaid card',
 7: 'Checking or savings account',
 8: 'Payday loan, title loan, or personal loan',
 9: 'Consumer Loan',
 10: 'Other financial service',
 11: 'Bank account or service',
 12: 'Money transfers',
 13: 'Prepaid card',
 14: 'Virtual currency'}

In [ ]:
categories = data['Product'].unique()
n_categories = len(categories)

In [ ]:
# Split into train and test datasets
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3, random_state=777) # random_state to reproduce results

In [ ]:
# Import tokenizer from HuggingFace
tokenizer = XLMTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
X_train = xlm_encode(X_train, tokenizer)
X_test = xlm_encode(X_test, tokenizer)

y_train = np.asarray(y_train, dtype='int32')
y_test = np.asarray(y_test, dtype='int32')

In [ ]:
X_test

{'input_mask': array([[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int32),
 'input_type_ids': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int32),
 'input_word_ids': array([[    0, 16152,  9426, ...,   449,    16,     2],
        [    0,  1404,  3999, ...,     1,     1,     1],
        [    0,  1822,   425, ...,     1,     1,     1],
        ...,
        [    0,  1822,   284, ...,     1,     1,     1],
        [    0,  2158,  2942, ...,     1,     1,     1],
        [    0,  8400,  4885, ...,     1,     1,     1]], dtype=int32)}

In [ ]:
def build_model(n_categories):
    with strategy.scope():
        input_word_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_word_ids')
        input_mask = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_mask')
        input_type_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_type_ids')

        # Import xlm model from HuggingFace
        xlm_model = TFXLMModel.from_pretrained(MODEL_NAME)
        x = xlm_model(input_word_ids, attention_mask=input_mask, token_type_ids=input_type_ids)

        # Huggingface transformers have multiple outputs, embeddings are the first one,
        # so let's slice out the first position
        x = x[0]

        x = tf.keras.layers.Dropout(0.1)(x)
        x = tf.keras.layers.Flatten()(x)
        x = tf.keras.layers.Dense(256, activation='relu')(x)
        x = tf.keras.layers.Dense(n_categories, activation='softmax')(x)

        model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=x)
        model.compile(
            optimizer=tf.keras.optimizers.Adam(lr=1e-5),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])

        return model

In [ ]:
with strategy.scope():
    model = build_model(n_categories)
    model.summary()

Some layers from the model checkpoint at xlm-mlm-ende-1024 were not used when initializing TFXLMModel: ['pred_layer_._proj']
- This IS expected if you are initializing TFXLMModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLMModel were initialized from the model checkpoint at xlm-mlm-ende-1024.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMModel for predictions without further training.


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_type_ids (InputLayer)     [(None, 256)]        0                                            
__________________________________________________________________________________________________
tfxlm_model_1 (TFXLMModel)      TFBaseModelOutput(la 142357504   input_word_ids[0][0]             
                                                                 input_mask[0][0]           

In [ ]:
with strategy.scope():
    print('Training...')
    history = model.fit(X_train,
                        y_train,
                        epochs=EPOCHS,
                        batch_size=BATCH_SIZE,
                        verbose=1,
                        validation_data=(X_test, y_test))

Training...
Epoch 1/3


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 256) dtype=int32>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 256) dtype=int32>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None, 256) dtype=int32>, <tf.Tensor 'cond_8/Identity_3:0' shape=(None,) dtype=int32>]


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 256) dtype=int32>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 256) dtype=int32>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None, 256) dtype=int32>, <tf.Tensor 'cond_8/Identity_3:0' shape=(None,) dtype=int32>]


12475/12475 [==============================] - ETA: 0s - loss: 0.7890 - accuracy: 0.7391

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 256) dtype=int32>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 256) dtype=int32>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None, 256) dtype=int32>, <tf.Tensor 'cond_8/Identity_3:0' shape=(None,) dtype=int32>]


12475/12475 [==============================] - 1439s 111ms/step - loss: 0.7890 - accuracy: 0.7391 - val_loss: 0.5885 - val_accuracy: 0.8024
Epoch 2/3
12475/12475 [==============================] - 1366s 110ms/step - loss: 0.5649 - accuracy: 0.8092 - val_loss: 0.5334 - val_accuracy: 0.8195
Epoch 3/3
12475/12475 [==============================] - 1368s 110ms/step - loss: 0.5025 - accuracy: 0.8280 - val_loss: 0.5041 - val_accuracy: 0.8290
